In [3]:
!pip install tensorflow_datasets

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 1.7 MB/s eta 0:00:0000:0100:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 2.5 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 2.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.9/152.9 kB 3.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 3.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.1/229.1 kB 3.0 MB/s eta 0:00:0000:010:01
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21486 sha256=dbbadd0858fc5d6df2c76c6e5dd4b116dab8255dfb3d32fc2a2327576ca70ce0
  Stored in directory: /root/.cache/pip/wheels/54/aa/01/724885182f93150035a2a91bce34a12877e8067a97baaf5dc8
Successfully built promise
  Attempting uninstall: protobuf
    Found existing installation: protob

In [4]:
!python --version

Python 3.8.10


In [46]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

In [6]:
tfds.__version__

'4.9.2'

In [8]:
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompletePTGM3Y/imdb_reviews-train.tfrecord…

...

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompletePTGM3Y/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompletePTGM3Y/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


2024-04-22 13:17:22.885330: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-22 13:17:22.896691: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-22 13:17:22.896861: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-22 13:17:22.900704: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the app

In [10]:
train_data, test_data = imdb["train"], imdb["test"]

In [27]:
train_data[0]

TypeError: 'PrefetchDataset' object is not subscriptable

In [28]:
training_sentences=[]
training_labels=[]

testing_sentences=[]
testing_labels=[]

for s, l in train_data:
    training_sentences.append(s.numpy().decode("utf8"))
    training_labels.append(l.numpy())
    
for s, l in test_data:
    testing_sentences.append(s.numpy().decode("utf8"))
    testing_labels.append(l.numpy())

In [29]:
len(training_sentences), len(testing_sentences)

(25000, 25000)

In [30]:
len(training_labels), len(testing_labels)

(25000, 25000)

In [31]:
training_sentences[0], training_labels[0]

("This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
 0)

### Tokenizing Training Sentences

In [33]:
vocab_size=10000
embedding_dim=16
max_len=120
trunc_type="post"
oov_tok="<OOV>"

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer=Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index=tokenizer.word_index
sequences=tokenizer.texts_to_sequences(training_sentences)
padded=pad_sequences(sequences, maxlen=max_len, truncating=trunc_type)

In [35]:
len(word_index), word_index

(88583,
 {'<OOV>': 1,
  'the': 2,
  'and': 3,
  'a': 4,
  'of': 5,
  'to': 6,
  'is': 7,
  'br': 8,
  'in': 9,
  'it': 10,
  'i': 11,
  'this': 12,
  'that': 13,
  'was': 14,
  'as': 15,
  'for': 16,
  'with': 17,
  'movie': 18,
  'but': 19,
  'film': 20,
  'on': 21,
  'not': 22,
  'you': 23,
  'are': 24,
  'his': 25,
  'have': 26,
  'he': 27,
  'be': 28,
  'one': 29,
  'all': 30,
  'at': 31,
  'by': 32,
  'an': 33,
  'they': 34,
  'who': 35,
  'so': 36,
  'from': 37,
  'like': 38,
  'her': 39,
  'or': 40,
  'just': 41,
  'about': 42,
  "it's": 43,
  'out': 44,
  'if': 45,
  'has': 46,
  'some': 47,
  'there': 48,
  'what': 49,
  'good': 50,
  'more': 51,
  'when': 52,
  'very': 53,
  'up': 54,
  'no': 55,
  'time': 56,
  'she': 57,
  'even': 58,
  'my': 59,
  'would': 60,
  'which': 61,
  'only': 62,
  'story': 63,
  'really': 64,
  'see': 65,
  'their': 66,
  'had': 67,
  'can': 68,
  'were': 69,
  'me': 70,
  'well': 71,
  'than': 72,
  'we': 73,
  'much': 74,
  'been': 75,
  'bad':

In [38]:
training_sentences[0]

"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."

In [37]:
sequences[0]

[12,
 14,
 33,
 425,
 392,
 18,
 90,
 28,
 1,
 9,
 32,
 1366,
 3585,
 40,
 486,
 1,
 197,
 24,
 85,
 154,
 19,
 12,
 213,
 329,
 28,
 66,
 247,
 215,
 9,
 477,
 58,
 66,
 85,
 114,
 98,
 22,
 5675,
 12,
 1322,
 643,
 767,
 12,
 18,
 7,
 33,
 400,
 8170,
 176,
 2455,
 416,
 2,
 89,
 1231,
 137,
 69,
 146,
 52,
 2,
 1,
 7577,
 69,
 229,
 66,
 2933,
 16,
 1,
 2904,
 1,
 1,
 1479,
 4940,
 3,
 39,
 3900,
 117,
 1584,
 17,
 3585,
 14,
 162,
 19,
 4,
 1231,
 917,
 7917,
 9,
 4,
 18,
 13,
 14,
 4139,
 5,
 99,
 145,
 1214,
 11,
 242,
 683,
 13,
 48,
 24,
 100,
 38,
 12,
 7181,
 5515,
 38,
 1366,
 1,
 50,
 401,
 11,
 98,
 1197,
 867,
 141,
 10]

### Tokenizing Testing Sentences

In [39]:
testing_sequences=tokenizer.texts_to_sequences(testing_sentences)
testing_padded=pad_sequences(testing_sequences, maxlen=max_len)

### Train Data

In [58]:
tf.config.list_logical_devices(), tf.config.list_physical_devices()

([LogicalDevice(name='/device:CPU:0', device_type='CPU'),
  LogicalDevice(name='/device:GPU:0', device_type='GPU')],
 [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
  PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')])

#### Model 1

In [51]:
model_1 = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_len),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")
], name="model_1")

model_1.compile(
    loss=tf.keras.losses.binary_crossentropy,
    optimizer="adam",
    metrics=["accuracy"],
)

model_1.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 120, 16)           160000    
                                                                 
 flatten_3 (Flatten)         (None, 1920)              0         
                                                                 
 dense_6 (Dense)             (None, 6)                 11526     
                                                                 
 dense_7 (Dense)             (None, 1)                 7         
                                                                 
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [52]:
%%time
history_1 = model_1.fit(
    padded,
    np.array(training_labels),
    epochs=10,
    validation_data=(testing_padded, np.array(testing_labels))
)

Epoch 1/10
782/782 [==============================] - 2s 2ms/step - loss: 0.4958 - accuracy: 0.7414 - val_loss: 0.3561 - val_accuracy: 0.8446
Epoch 2/10
782/782 [==============================] - 2s 2ms/step - loss: 0.2451 - accuracy: 0.9061 - val_loss: 0.3551 - val_accuracy: 0.8464
Epoch 3/10
782/782 [==============================] - 2s 2ms/step - loss: 0.1029 - accuracy: 0.9730 - val_loss: 0.4324 - val_accuracy: 0.8335
Epoch 4/10
782/782 [==============================] - 2s 2ms/step - loss: 0.0302 - accuracy: 0.9959 - val_loss: 0.5090 - val_accuracy: 0.8306
Epoch 5/10
782/782 [==============================] - 2s 2ms/step - loss: 0.0118 - accuracy: 0.9985 - val_loss: 0.5667 - val_accuracy: 0.8304
Epoch 6/10
782/782 [==============================] - 2s 2ms/step - loss: 0.0046 - accuracy: 0.9994 - val_loss: 0.6420 - val_accuracy: 0.8263
Epoch 7/10
782/782 [==============================] - 2s 2ms/step - loss: 0.0016 - accuracy: 0.9999 - val_loss: 0.6754 - val_accuracy: 0.8291
Epoch 

In [55]:
model_2 = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_len),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")
], name="model_2")

model_2.compile(
    loss=tf.keras.losses.binary_crossentropy,
    optimizer="adam",
    metrics=["accuracy"],
)

model_2.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 120, 16)           160000    
                                                                 
 global_average_pooling1d_1   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_10 (Dense)            (None, 6)                 102       
                                                                 
 dense_11 (Dense)            (None, 1)                 7         
                                                                 
Total params: 160,109
Trainable params: 160,109
Non-trainable params: 0
_________________________________________________________________


In [56]:
%%time
history_2 = model_2.fit(
    padded,
    np.array(training_labels),
    epochs=10,
    validation_data=(testing_padded, np.array(testing_labels))
)

Epoch 1/10
782/782 [==============================] - 2s 2ms/step - loss: 0.5634 - accuracy: 0.7387 - val_loss: 0.4010 - val_accuracy: 0.8374
Epoch 2/10
782/782 [==============================] - 2s 2ms/step - loss: 0.3431 - accuracy: 0.8589 - val_loss: 0.3353 - val_accuracy: 0.8568
Epoch 3/10
782/782 [==============================] - 2s 2ms/step - loss: 0.2777 - accuracy: 0.8885 - val_loss: 0.3269 - val_accuracy: 0.8590
Epoch 4/10
782/782 [==============================] - 2s 2ms/step - loss: 0.2410 - accuracy: 0.9067 - val_loss: 0.3389 - val_accuracy: 0.8551
Epoch 5/10
782/782 [==============================] - 2s 2ms/step - loss: 0.2143 - accuracy: 0.9192 - val_loss: 0.3561 - val_accuracy: 0.8496
Epoch 6/10
782/782 [==============================] - 2s 2ms/step - loss: 0.1927 - accuracy: 0.9298 - val_loss: 0.3779 - val_accuracy: 0.8459
Epoch 7/10
782/782 [==============================] - 2s 2ms/step - loss: 0.1751 - accuracy: 0.9381 - val_loss: 0.4107 - val_accuracy: 0.8395
Epoch 